In [2]:
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

# from lofo import LOFOImportance, Dataset, plot_importance
# from lightgbm import LGBMClassifier, LGBMRegressor

import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# import data
# data_df_1 = pd.read_csv("../../data/training_validation_1.csv")
data_df_2 = pd.read_csv("../../data/training_validation_2.csv")
# train_df = pd.concat([data_df_1, data_df_2], axis=0) # Merge data frames
train_df = data_df_2.filter(regex="vCnt|value")

# del train_df['class']
# del train_df['run']

In [4]:
train_df.head()

,CpuTemperature_value,DurationPickToPick_vCnt,DurationPickToPick_value,DurationRobotFromFeederToTestBench_vCnt,DurationRobotFromFeederToTestBench_value,DurationRobotFromTestBenchToFeeder_vCnt,DurationRobotFromTestBenchToFeeder_value,DurationTestBenchClosed_vCnt,DurationTestBenchClosed_value,EPOSCurrent_vCnt,...,TotalCpuLoadNormalized_value,TotalMemoryConsumption_value,Vacuum_vCnt,Vacuum_value,VacuumFusePicked_vCnt,VacuumFusePicked_value,VacuumValveClosed_vCnt,VacuumValveClosed_value,ValidFrame_vCnt,ValidFrameOptrisPIIRCamera_vCnt
0,59.050,1,3.215000,1,0.808000,1,0.797000,1,0.116000,48,...,71.446895,25570.854297,55,-0.466940,15,-0.331425,8,-0.611011,114.0,270.0
1,58.775,3,3.171333,3,0.787333,3,0.831667,3,0.112333,76,...,68.471328,25602.458984,90,-0.386998,46,-0.291387,24,-0.609363,113.0,270.0
2,58.825,3,3.122667,4,0.791500,3,0.796000,4,0.108750,77,...,67.776865,25581.930859,90,-0.397958,51,-0.311627,20,-0.655678,113.0,270.0
3,59.500,3,3.223333,3,0.807667,3,0.826333,3,0.106333,77,...,79.661067,25652.367578,92,-0.478855,47,-0.452090,26,-0.624444,114.0,271.0
4,60.525,3,3.214000,3,0.784000,3,0.801000,3,0.107667,75,...,70.097779,25751.534375,89,-0.424315,46,-0.354547,24,-0.634388,114.0,272.0


In [48]:
value_df = train_df.filter(regex="value")
value_df.head()

,CpuTemperature_value,DurationPickToPick_value,DurationRobotFromFeederToTestBench_value,DurationRobotFromTestBenchToFeeder_value,DurationTestBenchClosed_value,EPOSCurrent_value,EPOSPosition_value,EPOSVelocity_value,FeederBackgroundIlluminationIntensity_value,FuseCycleDuration_value,...,SharpnessImage_value,SmartMotorPositionError_value,SmartMotorSpeed_value,Temperature_value,TemperatureThermoCam_value,TotalCpuLoadNormalized_value,TotalMemoryConsumption_value,Vacuum_value,VacuumFusePicked_value,VacuumValveClosed_value
0,59.050,3.215000,0.808000,0.797000,0.116000,-0.229167,0.000000,0.000000,NaN,NaN,...,NaN,-197.20,-314.4,26.128571,NaN,71.446895,25570.854297,-0.466940,-0.331425,-0.611011
1,58.775,3.171333,0.787333,0.831667,0.112333,-0.421053,19.934211,-3.934211,NaN,NaN,...,NaN,-331.25,-524.0,26.105715,25.210781,68.471328,25602.458984,-0.386998,-0.291387,-0.609363
2,58.825,3.122667,0.791500,0.796000,0.108750,14.298701,3.571429,-3.051948,144.707031,NaN,...,15.777344,-327.40,-524.0,26.122857,25.199283,67.776865,25581.930859,-0.397958,-0.311627,-0.655678
3,59.500,3.223333,0.807667,0.826333,0.106333,7.662338,16.441558,11.883117,NaN,NaN,...,NaN,-331.00,-524.0,26.133333,25.204275,79.661067,25652.367578,-0.478855,-0.452090,-0.624444
4,60.525,3.214000,0.784000,0.801000,0.107667,7.840000,16.906667,-7.453333,NaN,16.946809,...,NaN,-329.40,-524.0,26.111429,25.223598,70.097779,25751.534375,-0.424315,-0.354547,-0.634388


In [49]:
value_df = value_df.dropna()
value_df.shape

(1164, 36)

In [59]:
features = value_df.columns

In [60]:
import os

In [61]:
%%time
d = "values"
for f1 in features:
    print(f1)
    dir = os.path.join(d, f1)
    if not os.path.isdir(dir): os.mkdir(dir) 
    for f2 in features:
        fig = plt.figure(figsize=(10,8))
        plt.scatter(value_df[f1], value_df[f2], marker='.')
        plt.xlabel(f1)
        plt.ylabel(f2)
        plt.savefig(os.path.join(dir, f2) + ".png")
        plt.close()
#         break
#     break


CpuTemperature_value
DurationPickToPick_value
DurationRobotFromFeederToTestBench_value
DurationRobotFromTestBenchToFeeder_value
DurationTestBenchClosed_value
EPOSCurrent_value
EPOSPosition_value
EPOSVelocity_value
FeederBackgroundIlluminationIntensity_value
FuseCycleDuration_value
FuseHeatSlope_value
FuseHeatSlopeNOK_value
FuseHeatSlopeOK_value
FuseOutsideOperationalSpace_value
FusePicked_value
FuseTestResult_value
Humidity_value
IntensityTotalImage_value
IntensityTotalThermoImage_value
LightBarrierActiveTaskDuration1_value
LightBarrierPassiveTaskDuration1_value
NumberFuseDetected_value
NumberFuseEstimated_value
Pressure_value
ProcessCpuLoadNormalized_value
ProcessMemoryConsumption_value
SharpnessImage_value
SmartMotorPositionError_value
SmartMotorSpeed_value
Temperature_value
TemperatureThermoCam_value
TotalCpuLoadNormalized_value
TotalMemoryConsumption_value
Vacuum_value
VacuumFusePicked_value
VacuumValveClosed_value
Wall time: 5min 2s


In [53]:
count_df = train_df.filter(regex="vCnt")
count_df.head()

,DurationPickToPick_vCnt,DurationRobotFromFeederToTestBench_vCnt,DurationRobotFromTestBenchToFeeder_vCnt,DurationTestBenchClosed_vCnt,EPOSCurrent_vCnt,EPOSPosition_vCnt,EPOSVelocity_vCnt,ErrorFrame_vCnt,FeederAction1_vCnt,FeederAction2_vCnt,...,Pressure_vCnt,SharpnessImage_vCnt,SmartMotorPositionError_vCnt,SmartMotorSpeed_vCnt,TemperatureThermoCam_vCnt,Vacuum_vCnt,VacuumFusePicked_vCnt,VacuumValveClosed_vCnt,ValidFrame_vCnt,ValidFrameOptrisPIIRCamera_vCnt
0,1,1,1,1,48,48,48,0.0,0,0,...,71,0,5,5,0,55,15,8,114.0,270.0
1,3,3,3,3,76,76,76,0.0,0,0,...,98,0,4,4,3,90,46,24,113.0,270.0
2,3,4,3,4,77,77,77,0.0,0,0,...,81,2,5,5,1,90,51,20,113.0,270.0
3,3,3,3,3,77,77,77,0.0,0,0,...,39,0,5,5,3,92,47,26,114.0,271.0
4,3,3,3,3,75,75,75,0.0,0,0,...,40,0,5,5,3,89,46,24,114.0,272.0


In [54]:
count_df = count_df.dropna()
count_df.shape

(17975, 43)

In [62]:
features = count_df.columns

In [63]:
import os

In [64]:
%%time
d = "vcnts"
for f1 in features:
    print(f1)
    dir = os.path.join(d, f1)
    if not os.path.isdir(dir): os.mkdir(dir) 
    for f2 in features:
        fig = plt.figure(figsize=(10,8))
        plt.scatter(count_df[f1], count_df[f2], marker='.')
        plt.xlabel(f1)
        plt.ylabel(f2)
        plt.savefig(os.path.join(dir, f2) + ".png")
        plt.close()
#         break
#     break


DurationPickToPick_vCnt
DurationRobotFromFeederToTestBench_vCnt
DurationRobotFromTestBenchToFeeder_vCnt
DurationTestBenchClosed_vCnt
EPOSCurrent_vCnt
EPOSPosition_vCnt
EPOSVelocity_vCnt
ErrorFrame_vCnt
FeederAction1_vCnt
FeederAction2_vCnt
FeederAction3_vCnt
FeederAction4_vCnt
FeederBackgroundIlluminationIntensity_vCnt
FuseCycleDuration_vCnt
FuseHeatSlope_vCnt
FuseHeatSlopeNOK_vCnt
FuseHeatSlopeOK_vCnt
FuseIntoFeeder_vCnt
FuseOutsideOperationalSpace_vCnt
FusePicked_vCnt
FuseTestResult_vCnt
IntensityTotalImage_vCnt
IntensityTotalThermoImage_vCnt
LightBarrieActiveTaskDuration2_vCnt
LightBarrierActiveTaskDuration1_vCnt
LightBarrierActiveTaskDuration1b_vCnt
LightBarrierPassiveTaskDuration1_vCnt
LightBarrierPassiveTaskDuration1b_vCnt
LightBarrierPassiveTaskDuration2_vCnt
LightBarrierTaskDuration_vCnt
NumberEmptyFeeder_vCnt
NumberFuseDetected_vCnt
NumberFuseEstimated_vCnt
Pressure_vCnt
SharpnessImage_vCnt
SmartMotorPositionError_vCnt
SmartMotorSpeed_vCnt
TemperatureThermoCam_vCnt
Vacuum_vCnt